### Modelo preditivo de Placar Final

### **Introdução**
O presente Colab tem como objetivo apresentar o processo de desenvolvimento de um dos modelos preditivos candidatos para responder a pergunta: **"Qual será o placar final?"**

Durante a Sprint 4, dedicamos tempo principalmente em uma realização de nova análise e exploração de dados (com escolhas de novas features), e também testes de diferentes modelos preditivos, visando melhorar as métricas dos modelos nas previsões de número de gols do time da casa e de visitante em uma partida.

#### **1. Exploração de Dados**
Realizamos uma extensa análise dos dados disponíveis, incluindo uma limpeza inicial para remover duplicatas e valores nulos. Fizemos o merge das tabelas com informações sobre os times e as partidas, e selecionamos as variáveis mais relevantes para a modelagem. Além disso, utilizamos técnicas estatísticas e visualizações como matrizes de correlação e heatmaps para identificar as variáveis mais importantes relacionadas aos gols do time da casa e visitante.

#### **2. Testagem de Diversos Modelos**
Após preparar a base de dados, exploramos diversos modelos de classificação, realizando validação cruzada e demonstrando métricas para avaliação de desempenho de modelos.


**Observação:** esse NÃO foi o modelo escolhido, nem é o Colab que é usado como base de comparação de modelos para a Sprint 4. Está sendo apresentado para fins de demonstração de aprendizagem do grupo, como solicitado pela orientadora.



#### Configuração de ambiente de desenvolvimento

In [ ]:
# Instalar bibliotecas necessárias
!pip install pandas matplotlib seaborn scikit-learn

In [ ]:
# Importação de bibliotecas para análise de dados, visualização e modelagem

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, log_loss
import warnings

warnings.filterwarnings('ignore')


### **1. Exploração e Análise de Dados**





#### **1.1. Carregamento e limpeza de dados**

*   Remoção de duplicatas e nulos
*   Escolha de apenas de jogos completos
*   Descrição estatística


In [ ]:
# Carregar tabelas CSV com informações de times e partidas, para realizar o merge das tabelas mais relevantes para o "placar final"

teams1 = pd.read_csv('brazil-serie-a-teams-2024-to-2024-stats.csv')
teams2 = pd.read_csv('brazil-serie-a-teams2-2024-to-2024-stats.csv')
matches = pd.read_csv('brazil-serie-a-matches-2024-to-2024-stats.csv', sep=';')

In [ ]:
# Remover duplicatas e valores nulos em cada tabela
teams1.drop_duplicates(inplace=True)
teams1.dropna(inplace=True)

teams2.drop_duplicates(inplace=True)
teams2.dropna(inplace=True)

matches.drop_duplicates(inplace=True)
matches.dropna(subset=['status'], inplace=True)  # Remover apenas linhas com status nulo


In [ ]:
# Exibir uma descrição dos datasets, com informações estatísticas

print(matches.describe())
print(teams1.describe())
print(teams2.describe())


In [ ]:
# Filtra apenas jogos completos no dataset matches
matches = matches[matches['status'] == 'complete']

In [ ]:
# Mostra as primeiras linhas do dataset de partidas após filtragem
matches.head()

#### **1.2. Merge das tabelas**

*   Junção das tabelas baseado em team_name





In [ ]:
# Merge das tabelas teams1 e teams2
teams = pd.concat([teams1, teams2], axis=0).drop_duplicates(subset=['team_name'])

# Merge com a tabela matches baseado no nome do time
merged_df = matches.merge(teams, left_on='home_team_name', right_on='team_name', how='left')
merged_df = merged_df.merge(teams, left_on='away_team_name', right_on='team_name', how='left', suffixes=('_home', '_away'))


In [ ]:
# Mostra as 5 primeiras linhas do dataset de partidas após o merge de arquivos

merged_df.head()

In [ ]:
# Remove as colunas irrelevantes como timestamp, date_GMT, referee, e outras que não contribuem para a predição
columns_to_drop = ['timestamp', 'attendance', 'date_GMT', 'referee', 'stadium_name']

# Removendo essas colunas das três tabelas principais
merged_df = merged_df.drop(columns=columns_to_drop)


In [ ]:
# Mostra as 5 primeiras linhas do dataset de partidas depois de tirar essas colunas
merged_df

#### **1.3. Label Encoding (nomes de times)**
*   Codificação das features categóricas, nesse caso os nomes dos times `(home_team_name,	away_team_name)`




In [ ]:
# Cria um dicionário para organizar o nome dos times a um número único
team_mapping = {
    'Atlético GO': 1,
    'Botafogo': 2,
    'Atlético Mineiro': 3,
    'Atlético PR': 4,
    'Flamengo': 5,
    'Vasco da Gama': 6,
    'Bragantino': 7,
    'Criciúma': 8,
    'Cruzeiro': 9,
    'Cuiabá': 10,
    'Bahia': 11,
    'Juventude': 12,
    'Vitória': 13,
    'Fluminense': 14,
    'Fortaleza': 15,
    'Grêmio': 16,
    'Corinthians': 17,
    'Internacional': 18,
    'Palmeiras': 19,
    'São Paulo': 20
}

merged_df['home_team_name_encoded'] = merged_df['home_team_name'].map(team_mapping)
merged_df['away_team_name_encoded'] = merged_df['away_team_name'].map(team_mapping)

print(merged_df[['home_team_name', 'away_team_name', 'home_team_name_encoded', 'away_team_name_encoded']].head())


In [ ]:
# Função para remover colunas com muitos valores nulos
def clean_data(df, threshold=0.8):
    merged_df = df.dropna(thresh=df.shape[0] * (1 - threshold), axis=1)
    return merged_df

# Limpar o dataframe, mantendo colunas com menos de 80% de valores nulos
merged_df = clean_data(merged_df, threshold=0.8)

In [ ]:
# Selecionar apenas colunas numéricas
merged_df = merged_df.select_dtypes(include=['float64', 'int64'])
merged_df.head()

#### **1.4. Matriz de correlação (gols da casa e visitante)**

In [ ]:
# Calcula a matriz de correlação e a correlação dos gols da casa e do visitante

correlation_matrix = merged_df.corr()
home_goals_corr = correlation_matrix['home_team_goal_count'].sort_values(ascending=False)
away_goals_corr = correlation_matrix['away_team_goal_count'].sort_values(ascending=False)

# Exibe as 10 maiores correlações com os gols da casa e do visitante
home_goals_corr.head(10), away_goals_corr.head(10)


In [ ]:
# Mostra a matriz de correlação usando um heatmap

plt.figure(figsize=(14,10))
sns.heatmap(merged_df.corr(), annot=False, cmap='coolwarm', linewidths=0.5)
plt.title("Matriz de Correlação")
plt.show()


#### **1.5. Remoção de outliers**

In [ ]:
# Padroniza os dados para identificar outliers
scaler = StandardScaler()
scaled_data = scaler.fit_transform(merged_df.select_dtypes(include=[np.number]))

# Algoritmo para detecção de outliers
iso_forest = IsolationForest(contamination=0.05)
outliers = iso_forest.fit_predict(scaled_data)

# Filtra os outliers (onde outliers == 1 são os dados normais)
cleaned_df = merged_df[outliers == 1]


In [ ]:
# Mostra o dataframe após limpeza de outliers
cleaned_df

In [ ]:
# Apresenta graficamente a distribuição dos gols da casa após remover outliers

plt.figure(figsize=(10,6))
sns.boxplot(x=cleaned_df['home_team_goal_count'])
plt.title('Distribuição da Assistência (Sem Outliers)')
plt.show()


### **2. Testagem de modelos preditivos e avaliação de métricas**

Nessa etapa vamos testar uma variedade de modelos de classificação e regressão, incluindo desde métodos mais simples, como Regressão Logística, até algoritmos avançados como XGBoost, LightGBM, Gradient Boosting e Random Forest.

Pra cada modelo, realizamos validação cruzada para garantir a consistência dos resultados.

Exploramos também métricas variadas para avaliar o desempenho, como Acurácia, Precisão, Recall, F1-Score, AUC-ROC e Log Loss.

In [ ]:
# Separar as features (X) e os targets (y) para gols da casa e do visitante
X =  merged_df.drop(['home_team_goal_count', 'away_team_goal_count'], axis=1)
y_home = merged_df['home_team_goal_count']
y_away =  merged_df['away_team_goal_count']

# Divisão do conjunto de dados em treino e teste
X_train, X_test, y_train_home, y_test_home = train_test_split(X, y_home, test_size=0.3, random_state=42)
X_train, X_test, y_train_away, y_test_away = train_test_split(X, y_away, test_size=0.3, random_state=42)

# Normalização dos dados
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


#### **2.1. Foco Principal no Time da Casa:**

A partir desse momento, iremos analisar principalmente o desempenho dos modelos para a variável de "home_team_goal_count", buscando identificar quais algoritmos apresentavam melhor desempenho para prever o número de gols do time da casa.


**`Definição e Avaliação de Modelos:`**

  Nesta seção, definimos e avaliamos diferentes modelos de classificação para prever o **número de gols do time da casa.**
O processo envolve a configuração de um conjunto de modelos, aplicação de validação cruzada e cálculo das principais métricas de desempenho.


#### **2.2. Testes em modelos, métricas e cross-validation**

  **`Modelos testados:`**

- Logistic Regression
- K-Nearest Neighbors (KNN)
- Support Vector Machine (SVM)
- Decision Tree
- Random Forest
- Gradient Boosting
- AdaBoost
- XGBoost
- LightGBM
- Extra Trees
- Gaussian Naive Bayes
- Linear Discriminant Analysis (LDA)
- Quadratic Discriminant Analysis (QDA)

**`Validação Cruzada:`**

Para avaliar o desempenho de cada modelo, utilizamos um processo de **validação cruzada** com 5 folds (`n_folds = 5`). Este processo divide os dados em 5 partes e treina o modelo em 4 partes, utilizando a parte restante para avaliação.
Esse processo é repetido até que todos os subconjuntos tenham sido usados como conjunto de teste uma vez, resultando em uma média das métricas de desempenho.


In [ ]:
# Definir os modelos a serem testados com hiperparâmetros

model_dict = {
    'Logistic Regression': LogisticRegression(random_state=42),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Support Vector Machine': SVC(probability=True, random_state=42),
    'Decision Tree': DecisionTreeClassifier(random_state=42),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'AdaBoost': AdaBoostClassifier(random_state=42),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    'LightGBM': LGBMClassifier(random_state=42),
    'Extra Trees': ExtraTreesClassifier(random_state=42),
    'Gaussian Naive Bayes': GaussianNB(),
    'Linear Discriminant Analysis': LinearDiscriminantAnalysis(),
    'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis()
}

cv_results_dict_home = {
    'Modelo': [],
    'Acurácia': [],
    'Precisão': [],
    'Recall': [],
    'F1-Score': [],
    'AUC-ROC': [],
    'Log Loss': []
}

# Número de folds para validação cruzada
n_folds = 5

# Treinar e avaliar cada modelo
for model_name, model in model_dict.items():
    # Validação cruzada
    cv_results = cross_validate(model, X_train_scaled, y_train_home, cv=n_folds,
                                scoring=['accuracy', 'precision', 'recall', 'f1', 'roc_auc', 'neg_log_loss'])

    # Armazenar resultados
    cv_results_dict_home['Modelo'].append(model_name)
    cv_results_dict_home['Acurácia'].append(cv_results['test_accuracy'].mean())
    cv_results_dict_home['Precisão'].append(cv_results['test_precision'].mean())
    cv_results_dict_home['Recall'].append(cv_results['test_recall'].mean())
    cv_results_dict_home['F1-Score'].append(cv_results['test_f1'].mean())
    cv_results_dict_home['AUC-ROC'].append(cv_results['test_roc_auc'].mean())
    cv_results_dict_home['Log Loss'].append(-cv_results['test_neg_log_loss'].mean())

# Convertendo resultados para DataFrame
cv_results_df_home = pd.DataFrame(cv_results_dict_home)

# Ordenar por Log Loss
cv_results_df_home = cv_results_df_home.sort_values(by='Log Loss', ascending=True)

# Exibir resultados
cv_results_df_home


Durante a fase de validação cruzada dos modelos, observamos que várias métricas, como Precisão, Recall, F1-Score, AUC-ROC e Log Loss, retornaram valores `NaN` (não numéricos). Esse comportamento foi verificado em diferentes modelos, como Logistic Regression, K-Nearest Neighbors, SVM, entre outros.

Entendemos que esse problema pode ter ocorrido devido a algumas razões, como:

**Desequilíbrio de Classes**:

Em alguns casos, a classe preditiva não estava presente em alguns folds da validação cruzada. Isso resulta em métricas como Precisão e Recall retornando valores indefinidos, pois não há exemplos positivos ou negativos suficientes em um determinado fold para o cálculo dessas métricas.

**Modelos x Métricas**:

Algumas métricas, como AUC-ROC e Log Loss, requerem que o modelo retorne probabilidades, e certos modelos não possuem o suporte adequado para o método `predict_proba()` ou `decision_function()`, o que impede o cálculo correto dessas métricas.


In [ ]:
print("Resultados para Gols do Time da Casa")
print(cv_results_df_home)

#### **2.3. Aplicação do melhor modelo**

Foi escolhido o modelo `Linear Discriminant Analysis (LDA)`, que é uma técnica de classificação multi-classe, para fazer as predições dos gols do time da casa. Esse modelo tenta encontrar a combinação linear de variáveis que melhor separa as classes e gera uma previsão para cada amostra de teste sobre quantos gols o time da casa fez.

A avaliação do modelo é feita através da métrica de acurácia usando accuracy_score, que compara as predições do modelo (y_pred_home) com os valores reais de gols do time da casa (y_test_home), sendo, nesse momento, de 58%.


In [ ]:
best_model_home = LinearDiscriminantAnalysis()
best_model_home.fit(X_train_scaled, y_train_home)

# Fazer predições
y_pred_home = best_model_home.predict(X_test_scaled)

# Avaliar as predições
accuracy_home = accuracy_score(y_test_home, y_pred_home)

print(f"Acurácia (Gols Casa): {accuracy_home}")


### **Conclusão**

Embora alguns modelos, como Linear Discriminant Analysis (escolhido como `best_model` (o melhor modelo) e Random Forest tenham mostrado um **desempenho aceitável** em métricas específicas, no geral, a variação de resultados entre os diferentes algoritmos foi grande, o que comprometeu a confiança nas previsões.

Além disso, devido ao o tempo limitado disponível para a Sprint, **decidimos utilizar outro modelo de Placar Final** (melhor ajustado) para predição dos gols do time da casa e time visitante.

Entendemos que seguindo principalmente uma abordagem de classificação exigiria ajustes significativos nos dados (como balanceamento de classes).

Por outro lado, temos outros **modelos candidatos, de regressão principalmente,** para responder essa mesma pergunta (que serão, portanto, utilizados para submissão do artefato e estará documentado como modelo principal para essa pergunta).

Sendo assim, o conteúdo do presente documento termina aqui, nós optamos por focar em outro modelo pra placar final, que apresentou métricas mais confiáveis durante a validação e **produziu comparações melhor fundamentadas** e maior explicabilidade.